In [1]:
# Semantic analyis of Twitter data

In [2]:
import pandas as pd
import nltk
import os
from textblob import TextBlob
import re

In [3]:
#CSV file with extracted tweets for CWC19Final hash-tag during before the event
file_1 = pd.read_csv("D:\\LJMU\\Project Topics\\Data\\Cleansed Data\\output_got_beforematch.csv",error_bad_lines=False,delimiter=";")
#CSV file with extracted tweets for EngVsNZ hash-tag during before the event
file_2 = pd.read_csv("D:\\LJMU\\Project Topics\\Data\\Cleansed Data\\output_got_ENGvsNZ.csv",error_bad_lines=False, delimiter =";", quotechar='\"')
#CSV file with extracted tweets for NZVsENG hash-tag during before the event
file_3 = pd.read_csv("D:\\LJMU\\Project Topics\\Data\\Cleansed Data\\output_got_NZvsENG.csv",error_bad_lines=False,delimiter=";")


#Combine the dataframes
interim = file_1.append(file_2)
combine = interim.append(file_3)

len(combine)


b'Skipping line 13: expected 10 fields, saw 11\nSkipping line 365: expected 10 fields, saw 11\nSkipping line 505: expected 10 fields, saw 11\nSkipping line 1188: expected 10 fields, saw 11\nSkipping line 1195: expected 10 fields, saw 11\nSkipping line 1202: expected 10 fields, saw 11\nSkipping line 1321: expected 10 fields, saw 11\nSkipping line 1451: expected 10 fields, saw 11\n'
c:\users\amitb\pycharmprojects\cricwintweet\venv\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


4910

In [4]:
combine.head(10)


,date,favorites,geo,hashtags,id,mentions,permalink,"permalink,,,,,,,,",retweets,text,username
0,2019-07-14 05:29,5,NaN,# # # #,1150193174633164801,@ @ @ @ @ @ @,https://twitter.com/msarimakhtar/status/115019...,NaN,0,Go England @ England @ englandcricket @ TheBar...,NaN
1,2019-07-14 05:29,0,NaN,# # #,1150193117254938624,NaN,https://twitter.com/Mohit9255/status/115019311...,NaN,0,# NZvENG # CWC19Final # CWC19pic.twitter.com/g...,NaN
2,2019-07-14 05:29,0,NaN,#,1150193066722189313,NaN,https://twitter.com/WooderzJames/status/115019...,NaN,0,Anyone else just ridiculously excited for the ...,NaN
3,2019-07-14 05:29,0,NaN,# # #,1150192971746304006,NaN,https://twitter.com/merichai/status/1150192971...,NaN,0,Supporting Kiwis # NZvENG # CWC19 # CWC19Final,NaN
4,2019-07-14 05:27,0,NaN,#,1150192673258524672,@,https://twitter.com/KeyurJoshi_/status/1150192...,NaN,0,# CWC19Final England and New Zealand final is ...,NaN
5,2019-07-14 05:26,5,NaN,# #,1150192244449824769,@ @ @ @ @ @ @,https://twitter.com/msarimakhtar/status/115019...,NaN,1,Congratulations to the new queen of @ Wimbledo...,NaN
6,2019-07-14 05:26,2,NaN,# #,1150192181254094848,NaN,https://twitter.com/IBT_Sports/status/11501921...,NaN,2,ICC 2019 World Cup final: 5 Biggest duels to w...,NaN
7,2019-07-14 05:25,0,NaN,# # # # # # # # # # # # # # # # # # # # # #,1150191956888305664,NaN,https://twitter.com/MMustafa2027pti/status/115...,NaN,0,Who Will Win # ICC # Cricket # World # CUP 201...,NaN
8,2019-07-14 05:24,13,NaN,# # #,1150191781830463488,@,https://twitter.com/JamieWall2/status/11501917...,NaN,0,"Go get 'em, @ BLACKCAPS. # CWC19 # CWC19Final ...",NaN
9,2019-07-14 05:24,232,NaN,#,1150191706928746497,NaN,https://twitter.com/bhattimajid/status/1150191...,NaN,5,"# CWC19Final Nice click, Thank you so much Kam...",NaN


In [5]:
#Remove duplicates from the dataset
combine = combine.drop_duplicates()

len(combine)

4860

In [6]:
# Removing redundant columns
combine = combine.drop(['hashtags'],axis=1)
combine = combine.drop(['geo'],axis=1)
combine = combine.drop(['mentions'],axis=1)
combine = combine.drop(['permalink'],axis=1)
combine = combine.drop(['permalink,,,,,,,,'],axis=1)
combine = combine.drop(['username'],axis=1)

#combine = combine.drop(index=[7,8],axis=1)

In [7]:
combine.head(10)

,date,favorites,id,retweets,text
0,2019-07-14 05:29,5,1150193174633164801,0,Go England @ England @ englandcricket @ TheBar...
1,2019-07-14 05:29,0,1150193117254938624,0,# NZvENG # CWC19Final # CWC19pic.twitter.com/g...
2,2019-07-14 05:29,0,1150193066722189313,0,Anyone else just ridiculously excited for the ...
3,2019-07-14 05:29,0,1150192971746304006,0,Supporting Kiwis # NZvENG # CWC19 # CWC19Final
4,2019-07-14 05:27,0,1150192673258524672,0,# CWC19Final England and New Zealand final is ...
5,2019-07-14 05:26,5,1150192244449824769,1,Congratulations to the new queen of @ Wimbledo...
6,2019-07-14 05:26,2,1150192181254094848,2,ICC 2019 World Cup final: 5 Biggest duels to w...
7,2019-07-14 05:25,0,1150191956888305664,0,Who Will Win # ICC # Cricket # World # CUP 201...
8,2019-07-14 05:24,13,1150191781830463488,0,"Go get 'em, @ BLACKCAPS. # CWC19 # CWC19Final ..."
9,2019-07-14 05:24,232,1150191706928746497,5,"# CWC19Final Nice click, Thank you so much Kam..."


In [8]:
#Data cleaning starts from here. Processing of text column

#combine['Processed Text'] = re.sub('https\:\/\/[\w\d.]','',combine['text'])

combine['Processed'] = combine.apply(lambda row: re.sub('http?\S+',"",row.text), axis=1)
combine['Processed'] = combine.apply(lambda row: re.sub('https\S+',"",row.Processed), axis=1)

# Remove pic.twitter.com
combine['Processed'] = combine.apply(lambda row: re.sub('pic.twitter.com\\S+',"",row.Processed), axis=1)

In [9]:
combine.head(20)

,date,favorites,id,retweets,text,Processed
0,2019-07-14 05:29,5,1150193174633164801,0,Go England @ England @ englandcricket @ TheBar...,Go England @ England @ englandcricket @ TheBar...
1,2019-07-14 05:29,0,1150193117254938624,0,# NZvENG # CWC19Final # CWC19pic.twitter.com/g...,# NZvENG # CWC19Final # CWC19
2,2019-07-14 05:29,0,1150193066722189313,0,Anyone else just ridiculously excited for the ...,Anyone else just ridiculously excited for the ...
3,2019-07-14 05:29,0,1150192971746304006,0,Supporting Kiwis # NZvENG # CWC19 # CWC19Final,Supporting Kiwis # NZvENG # CWC19 # CWC19Final
4,2019-07-14 05:27,0,1150192673258524672,0,# CWC19Final England and New Zealand final is ...,# CWC19Final England and New Zealand final is ...
5,2019-07-14 05:26,5,1150192244449824769,1,Congratulations to the new queen of @ Wimbledo...,Congratulations to the new queen of @ Wimbledo...
6,2019-07-14 05:26,2,1150192181254094848,2,ICC 2019 World Cup final: 5 Biggest duels to w...,ICC 2019 World Cup final: 5 Biggest duels to w...
7,2019-07-14 05:25,0,1150191956888305664,0,Who Will Win # ICC # Cricket # World # CUP 201...,Who Will Win # ICC # Cricket # World # CUP 201...
8,2019-07-14 05:24,13,1150191781830463488,0,"Go get 'em, @ BLACKCAPS. # CWC19 # CWC19Final ...","Go get 'em, @ BLACKCAPS. # CWC19 # CWC19Final ..."
9,2019-07-14 05:24,232,1150191706928746497,5,"# CWC19Final Nice click, Thank you so much Kam...","# CWC19Final Nice click, Thank you so much Kam..."


In [10]:
# Remove twitter handles
combine['Processed'] = combine.apply(lambda row: re.sub('\@ \w+',"",row.Processed), axis=1)

# Remove HashTags from the text field.
combine['Processed'] = combine.apply(lambda row: re.sub('\# \w+',"",row.Processed), axis=1)


In [11]:
combine.head(20)


,date,favorites,id,retweets,text,Processed
0,2019-07-14 05:29,5,1150193174633164801,0,Go England @ England @ englandcricket @ TheBar...,Go England …
1,2019-07-14 05:29,0,1150193117254938624,0,# NZvENG # CWC19Final # CWC19pic.twitter.com/g...,
2,2019-07-14 05:29,0,1150193066722189313,0,Anyone else just ridiculously excited for the ...,Anyone else just ridiculously excited for the ...
3,2019-07-14 05:29,0,1150192971746304006,0,Supporting Kiwis # NZvENG # CWC19 # CWC19Final,Supporting Kiwis
4,2019-07-14 05:27,0,1150192673258524672,0,# CWC19Final England and New Zealand final is ...,England and New Zealand final is the best out...
5,2019-07-14 05:26,5,1150192244449824769,1,Congratulations to the new queen of @ Wimbledo...,Congratulations to the new queen of and the v...
6,2019-07-14 05:26,2,1150192181254094848,2,ICC 2019 World Cup final: 5 Biggest duels to w...,ICC 2019 World Cup final: 5 Biggest duels to w...
7,2019-07-14 05:25,0,1150191956888305664,0,Who Will Win # ICC # Cricket # World # CUP 201...,Who Will Win 2019 England vs NewZealand ?...
8,2019-07-14 05:24,13,1150191781830463488,0,"Go get 'em, @ BLACKCAPS. # CWC19 # CWC19Final ...","Go get 'em, ."
9,2019-07-14 05:24,232,1150191706928746497,5,"# CWC19Final Nice click, Thank you so much Kam...","Nice click, Thank you so much Kamal Sharma"


In [12]:
# Tagging the dataset with team names and the player names which are appearing in the list first

NZ_tags = ['new zealand','nz','blackcaps','black caps','Martin' ,'Guptill', 'Henry Nicholls', 'Kane','Williamson', 'Ross','Taylor', 'Tom','Latham', 'James','Neesham', 'Colin de','Grandhomme', 'Mitchell','Santner', 'Matt Henry', 'Trent','Boult']

ENG_tags = ['england','english','Jason','Roy','Jonny','Bairstow','Joe','Root','Eoin','Morgan','Ben','Stokes','Jos','Buttler','Chris','Woakes','Liam','Plunkett','Jofra','Archer','Adil','Rashid','Mark','Wood','eng']   

In [13]:
# Create a function to tag the data

def create_tag(text, pos_eng = 999, pos_nz = 999):
    for x in NZ_tags:
        if re.search(x,text,re.IGNORECASE):
            pos_nz = re.search(x,text,re.IGNORECASE).start()
            break
      
    for x in ENG_tags:
        if re.search(x,text,re.IGNORECASE):
            pos_eng = re.search(x,text,re.IGNORECASE).start()
            break
            
    if pos_nz < pos_eng:
        return "NZ"
    elif pos_eng < pos_nz:
        return "ENG"
    else:
        return "Neutral"
    

In [14]:
combine['TeamTag'] = [create_tag(x) for x in combine['text']]

In [15]:
# Get the summary of the tags created.

summary = pd.pivot_table(combine, index = 'TeamTag',aggfunc ='count')
summary

,Processed,date,favorites,id,retweets,text
TeamTag,,,,,,
ENG,2765,2765,2765,2765,2765,2765
NZ,1480,1480,1480,1480,1480,1480
Neutral,615,615,615,615,615,615


In [16]:
combine.head(10)

,date,favorites,id,retweets,text,Processed,TeamTag
0,2019-07-14 05:29,5,1150193174633164801,0,Go England @ England @ englandcricket @ TheBar...,Go England …,ENG
1,2019-07-14 05:29,0,1150193117254938624,0,# NZvENG # CWC19Final # CWC19pic.twitter.com/g...,,NZ
2,2019-07-14 05:29,0,1150193066722189313,0,Anyone else just ridiculously excited for the ...,Anyone else just ridiculously excited for the ...,NZ
3,2019-07-14 05:29,0,1150192971746304006,0,Supporting Kiwis # NZvENG # CWC19 # CWC19Final,Supporting Kiwis,NZ
4,2019-07-14 05:27,0,1150192673258524672,0,# CWC19Final England and New Zealand final is ...,England and New Zealand final is the best out...,ENG
5,2019-07-14 05:26,5,1150192244449824769,1,Congratulations to the new queen of @ Wimbledo...,Congratulations to the new queen of and the v...,ENG
6,2019-07-14 05:26,2,1150192181254094848,2,ICC 2019 World Cup final: 5 Biggest duels to w...,ICC 2019 World Cup final: 5 Biggest duels to w...,ENG
7,2019-07-14 05:25,0,1150191956888305664,0,Who Will Win # ICC # Cricket # World # CUP 201...,Who Will Win 2019 England vs NewZealand ?...,ENG
8,2019-07-14 05:24,13,1150191781830463488,0,"Go get 'em, @ BLACKCAPS. # CWC19 # CWC19Final ...","Go get 'em, .",NZ
9,2019-07-14 05:24,232,1150191706928746497,5,"# CWC19Final Nice click, Thank you so much Kam...","Nice click, Thank you so much Kamal Sharma",Neutral
